# Logistic Regression

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

## Taking the AAPL for example to investigate the effect

#### Monthly

In [17]:
freq = 'Daily'
stock = 'AAPL'
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.values
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values
predictors

,pcf,PEG_trailing,dpr,npm,gpm,roa,roe,capital_ratio,de_ratio,cash_ratio,...,unemp,employ,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,6.793743,-0.118869,0.040186,0.222148,0.430765,0.193824,0.319423,-0.087769,0.103892,0.674998,...,9.795025,129887.174642,0.000000,214.010000,0.000000,0.000000,0.000000,50.505747,1.405796e+23,9.482004e+06
2010-01-05,4.187225,-0.053682,-0.075989,0.208720,0.448112,0.172678,0.274820,0.109338,0.698921,0.300487,...,9.795025,129887.174642,0.000000,214.059333,0.000000,0.000000,0.000000,59.261361,1.378231e+23,1.822908e+07
2010-01-06,3.948972,0.174516,0.122371,0.227693,0.433884,0.209830,0.291358,0.013588,0.281245,0.555570,...,9.795025,129887.174642,0.000000,213.647422,0.000000,0.000000,0.000000,66.624864,1.351207e+23,2.726522e+07
2010-01-07,6.621145,0.044860,0.073927,0.217652,0.408272,0.157871,0.250658,0.241239,0.261518,0.480624,...,9.795025,129887.174642,0.000000,213.238433,0.000000,0.000000,0.000000,64.905487,1.324713e+23,3.566595e+07
2010-01-08,6.615392,0.254345,0.079834,0.208321,0.411917,0.164579,0.223975,-0.117055,0.950571,0.537693,...,9.795025,129887.174642,0.000000,213.070642,0.000000,0.000000,0.000000,63.803522,1.298738e+23,4.303382e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-27,6.844910,-0.370283,0.117476,0.243125,0.394445,0.298552,0.361601,0.111357,0.694178,0.593535,...,3.334949,152178.876862,312.660714,310.922870,30883.645013,31539.108233,59.513412,0.000000,2.971600e+02,1.728001e+11
2020-01-28,8.496805,0.933175,-0.066304,0.203920,0.437381,0.188477,0.303203,0.097377,0.829619,0.556700,...,3.333884,152172.280972,314.039286,311.825154,31757.645013,31487.572259,67.412486,0.000000,2.971600e+02,1.751081e+11
2020-01-29,12.919763,0.408613,0.046701,0.207416,0.452347,0.344992,0.294698,0.179833,0.791039,0.780120,...,3.333221,152165.068670,315.550000,313.493800,32420.727035,31687.339021,68.465165,0.000000,3.048800e+02,1.774115e+11


In [18]:
clf = LogisticRegression().fit(X,y)
clf.score(X,y)

0.5301537248718959